In [1]:
import torch
import numpy as np 
import pickle
import pandas
import os
import glob

In [2]:
from dotenv import load_dotenv
load_dotenv()

os.getenv("DATA_PATH")

'D:\\\\Project\\\\AIProject\\\\AI_Data\\\\Phoenix_Data\\\\PHOENIX-2014-T-release-v3\\\\PHOENIX-2014-T'

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

os.getenv("DATA_PATH")

'D:\\\\Project\\\\AIProject\\\\AI_Data\\\\Phoenix_Data\\\\PHOENIX-2014-T-release-v3\\\\PHOENIX-2014-T'

In [4]:
# load the gloss_dict.npy
gloss_dict = np.load('..\\Information_dict\\gloss_dict.npy', allow_pickle=True).item()
gloss_dict

{'A': [1, 8],
 'A+Z': [2, 1],
 'AACHEN': [3, 3],
 'AB': [4, 76],
 'AB-JETZT': [5, 1],
 'AB-SO': [6, 1],
 'ABEND': [7, 495],
 'ABER': [8, 438],
 'ABFALLEN': [9, 1],
 'ABKUEHLEN': [10, 1],
 'ABSCHIED': [11, 1],
 'ABSCHNITT': [12, 1],
 'ABSINKEN': [13, 2],
 'ABWECHSELN': [14, 6],
 'ACH': [15, 15],
 'ACHT': [16, 160],
 'ACHTE': [17, 16],
 'ACHTHUNDERT': [18, 5],
 'ACHTUNG': [19, 14],
 'ACHTZEHN': [20, 96],
 'ACHTZIG': [21, 3],
 'AEHNLICH': [22, 21],
 'AENDERN': [23, 1],
 'AFRIKA': [24, 1],
 'AKTIV': [25, 2],
 'AKTUELL': [26, 3],
 'ALLE': [27, 11],
 'ALLGAEU': [28, 31],
 'ALLGEMEIN': [29, 1],
 'ALPEN': [30, 238],
 'ALPENRAND': [31, 1],
 'ALPENTAL': [32, 1],
 'ALS': [33, 15],
 'ALSO': [34, 3],
 'ALT': [35, 2],
 'AM': [36, 8],
 'AM-KUESTE': [37, 2],
 'AM-MEER': [38, 1],
 'AM-RAND': [39, 1],
 'AM-TAG': [40, 12],
 'AMERIKA': [41, 1],
 'AN': [42, 3],
 'ANDERE': [43, 20],
 'ANDERE-MOEGLICHKEIT': [44, 1],
 'ANDERS': [45, 5],
 'ANFANG': [46, 148],
 'ANGEMESSEN': [47, 3],
 'ANGENEHM': [48, 21],
 'AN

In [5]:
import pickle
import pandas as pd

# load the pickle file
train_info = pickle.load(open('..\\Information_dict\\train_info.pkl', 'rb'))

train_info[0]


{'fileid': '11August_2010_Wednesday_tagesschau-1',
 'folder': 'train\\11August_2010_Wednesday_tagesschau-1',
 'signer': 'Signer08',
 'label': 'JETZT WETTER MORGEN DONNERSTAG ZWOELF FEBRUAR',
 'num_frames': 86}

In [6]:
import numpy as np
# create a 3D list
list = [[[1, 2, 3], [4, 5, 6], [7, 8, 9]]]
# convert the list to a numpy array
arr = np.array(list)
arr.shape

(1, 3, 3)

In [7]:
import data_augmentation
from data_augmentation import *
import os
import cv2
import sys
import pdb
import six
import glob
import time
import torch
import random
import pandas
import warnings
import numpy as np
import pyarrow as pa
from PIL import Image
import torch.utils.data as data
import matplotlib.pyplot as plt
from torch.utils.data.sampler import Sampler
import pickle

class VideoDataset(data.Dataset):
    def __init__(self, prefix, gloss_dict, drop_ratio=1, num_gloss=-1, transform_mode= True, frame_interval= 1, image_scale= 1.0, kernel_size= 1, input_size= 224, mode= 'train', data_type= 'video'):
        self.mode= mode
        self.prefix= prefix
        self.transform_mode= True if mode == 'train' else False
        self.image_scale= image_scale
        global kernel_sizes
        kernel_sizes= kernel_size
        self.data_type= data_type
        self.dict= gloss_dict
        self.num_gloss= num_gloss
        self.input_size= input_size
        self.frame_interval= frame_interval
        self.drop_ratio= drop_ratio
        self.feature_prefix = f"{self.prefix}\\features\\fullFrame-256x256px\\{self.mode}"
        # load the pickle file
        self.inputs_list= pickle.load(open(f'..\\Information_dict\\{self.mode}_info.pkl', 'rb'))
        
    def __getitem__(self, index):
        if self.data_type == 'video':
            input_data, label, file_info = self.read_video(index)
            input_data, label = self.normalize(input_data, label)
            return input_data, torch.LongTensor(label), self.inputs_list[index]['label']
        elif self.data_type == 'feature':
            input_data, label = self.read_feature(index)
            return input_data, label, self.inputs_list[index]['label']
    
    def read_video(self, index):
        # load file info
        file_info =self.inputs_list[index]
        img_folder = self.prefix + '\\features\\fullFrame-256x256px\\' + file_info['folder']
        img_list = sorted(glob.glob(img_folder))
        img_list = img_list[int(torch.randint(0, self.frame_interval, [1]))::self.frame_interval]
        label_list = []
        for phase in file_info['label']:
            if phase == '':
                continue
            if phase in self.dict.keys():
                label_list.append(self.dict[phase][0])
        return [cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB) for img_path in img_list], label_list, file_info
    
    def read_feature(self, index):
        # load file info
        file_info = self.inputs_list[index]
        data = np.load(f'..\\features\\{self.mode}\\{file_info["fileid"]}.npy', allow_pickle= True)
        return data['features'], data['label']
    
    def normalize(self, video, label, file_id= None):
        video, label = self.data_augmentation(video, label, file_id)
        video = video.float() / 127.5 - 1
        return video, label
    
    def transform(self):
        if self.transform_mode:
            print("Apply training transform")
            return data_augmentation.Compose([
                # data_augmentation.RandomHorizontalFlip(),
                data_augmentation.RandomCrop(self.input_size, self.image_scale),
                data_augmentation.RandomHorizontalFlip(0.5),
                data_augmentation.Resize(self.image_scale),
                data_augmentation.ToTensor(),
                data_augmentation.TemporalRescale(0.2, self.frame_interval)
            ])
        else:
            print("Apply validation transform")
            return data_augmentation.Compose([
                data_augmentation.CenterCrop(self.input_size, self.image_scale),
                data_augmentation.Resize(self.image_scale),
                data_augmentation.ToTensor(),
            ])
            
    @staticmethod
    def collate_fn(batch):
        batch = [item for item in sorted(batch, key=lambda x: len(x[0]), reverse=True)]
        video, label, info = list(zip(*batch))
        
        left_pad = 0
        last_stride = 1
        total_stride = 1
        global kernel_sizes 
        for layer_idx, ks in enumerate(kernel_sizes):
            if ks[0] == 'K':
                left_pad = left_pad * last_stride 
                left_pad += int((int(ks[1])-1)/2)
            elif ks[0] == 'P':
                last_stride = int(ks[1])
                total_stride = total_stride * last_stride
        if len(video[0].shape) > 3:
            max_len = len(video[0])
            video_length = torch.LongTensor([np.ceil(len(vid) / total_stride) * total_stride + 2*left_pad for vid in video])
            right_pad = int(np.ceil(max_len / total_stride)) * total_stride - max_len + left_pad
            max_len = max_len + left_pad + right_pad
            padded_video = [torch.cat(
                (
                    vid[0][None].expand(left_pad, -1, -1, -1),
                    vid,
                    vid[-1][None].expand(max_len - len(vid) - left_pad, -1, -1, -1),
                )
                , dim=0)
                for vid in video]
            padded_video = torch.stack(padded_video)
        else:
            max_len = len(video[0])
            video_length = torch.LongTensor([len(vid) for vid in video])
            padded_video = [torch.cat(
                (
                    vid,
                    vid[-1][None].expand(max_len - len(vid), -1),
                )
                , dim=0)
                for vid in video]
            padded_video = torch.stack(padded_video).permute(0, 2, 1)
        label_length = torch.LongTensor([len(lab) for lab in label])
        if max(label_length) == 0:
            return padded_video, video_length, [], [], info
        else:
            padded_label = []
            for lab in label:
                padded_label.extend(lab)
            padded_label = torch.LongTensor(padded_label)
            return padded_video, video_length, padded_label, label_length, info

    def __len__(self):
        return len(self.inputs_list) - 1
    

In [8]:
prefix = os.getenv("DATA_PATH")
# load the numpy file
gloss_dict = np.load('..\\Information_dict\\gloss_dict.npy', allow_pickle= True).item()

feeder = VideoDataset(prefix= prefix, gloss_dict= gloss_dict)
dataloader = torch.utils.data.DataLoader(
    dataset=feeder,
    batch_size=1,
    shuffle=True,
    drop_last=True,
    num_workers=0,
)

In [10]:
# get the first batch
for batch in dataloader:
    video, video_length, label, label_length, info = batch
    print(video.shape)
    print(video_length)
    print(label)
    print(label_length)
    print(info)
    break

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
